In [1]:
!pip install yfinance

     |████████████████████████████████| 6.4 MB 33.3 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings
warnings.filterwarnings('ignore')

In [18]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2016-01-01'
df = yf.download('ETH-USD',start_date, today)
df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-02-04,2681.057617,2983.586914,2675.443848,2983.586914,2983.586914,18987223729
2022-02-05,2984.446045,3054.130127,2966.781006,3014.648193,3014.648193,13102093957
2022-02-06,3014.959717,3061.261230,2965.429932,3057.476074,3057.476074,9466018022
2022-02-07,3057.422119,3182.528320,3002.927002,3142.470703,3142.470703,15197063785
2022-02-08,3149.728760,3217.576172,3053.831055,3075.046143,3075.046143,17050627072


In [19]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1553 entries, 2017-11-09 to 2022-02-08
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1553 non-null   float64
 1   High       1553 non-null   float64
 2   Low        1553 non-null   float64
 3   Close      1553 non-null   float64
 4   Adj Close  1553 non-null   float64
 5   Volume     1553 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 84.9 KB


In [21]:
df.reset_index(inplace=True)
df.columns 

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [24]:
filtered_df = df[["Date", "Close"]]
new_names = {
    "Date": "ds", 
    "Close": "y",
}
filtered_df.rename(columns=new_names, inplace=True)

In [25]:
# plot the open price

x = filtered_df["ds"]
y = filtered_df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of Ethereum Open Price",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [38]:
filtered_df["date"] = pd.to_datetime(filtered_df['ds'])

In [58]:
train_df = filtered_df[(filtered_df['date'] > pd.Timestamp(2016,1,1)) & (filtered_df['date'] < pd.Timestamp(2021,8,1))] 
test_df = filtered_df[(filtered_df['date'] > pd.Timestamp(2021,8,1))] 

In [59]:
m = Prophet(
    seasonality_mode="multiplicative" 
)

m.fit(train_df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [60]:
future = m.make_future_dataframe(periods = 365//2)
future.tail()

,ds
1538,2022-01-25
1539,2022-01-26
1540,2022-01-27
1541,2022-01-28
1542,2022-01-29


In [61]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
1538,2022-01-25,3798.065617,3452.799346,4101.227410
1539,2022-01-26,3785.139242,3457.058034,4086.745611
1540,2022-01-27,3764.676371,3434.358673,4085.716723
1541,2022-01-28,3744.811662,3423.860698,4049.350586
1542,2022-01-29,3759.061811,3425.530684,4067.863353


In [ ]:
# next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

# forecast[forecast['ds'] == next_day]['yhat'].item()

In [63]:
plot_plotly(m, forecast)

In [49]:
#@title Plot Actual Price
# plot the open price

x = filtered_df["ds"]
y = filtered_df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of Ethereum Open Price",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [64]:
plot_components_plotly(m, forecast)